In [1]:
#필요모듈, 만든 함수 불러오기
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from layers import MultiLayer
from layers import Relu, Affine, SoftmaxWithLoss, Sigmoid
from collections import OrderedDict

In [2]:
def make_one(x):
    if x.ndim == 1: 
        t = np.zeros((x.size,np.unique(x).size))
        for i in range(t.shape[0]):
            t[i,x[i]]=1
        return t

In [3]:
# 학습 1:: 아이리스 꽃

In [5]:
from sklearn.datasets import load_iris
X = load_iris()['data']
y = load_iris()['target']
y = make_one(y)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.4)
X_val, X_test, y_val, y_test = train_test_split(X_test,y_test, test_size=.5)

input_size = X_train.shape[1]
hidden_size = [5]
output_size = y_train.shape[1]

model = MultiLayer(input_size,hidden_size,output_size)
epochs = 10 #에폭 알아서 설정해주기 그냥 오래 걸려서 10 해줌 
lr = 1e-3
model.fit_gd(epochs,lr,X_train,y_train,X_val,y_val)
model.accuracy(X_test,y_test)

epoch  0 :val_loss=========== 3.7711676400319436 val_acc:======== 0.36666666666666664


0.23333333333333334

In [ ]:
# 학습 2:: 숫자그림 #불러오는데 오래걸림여

In [6]:
mnist = fetch_openml('mnist_784')
X = mnist['data']
y = mnist['target']
X = X.astype(np.float32).values/255.
y = y.astype(np.int32).values
y = make_one(y)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.4)
X_val, X_test, y_val, y_test = train_test_split(X_test,y_test, test_size=.5)

input_size = X_train.shape[1]
hidden_size = [100,256,100]
output_size = y_train.shape[1]

model = MultiLayer(input_size,hidden_size,output_size)
epochs = 10
lr = 1e-3
model.fit_gd(epochs,lr,X_train,y_train,X_val,y_val)
model.accuracy(X_test,y_test)

epoch  0 :val_loss=========== 13.854447649733846 val_acc:======== 0.14


0.47185714285714286

In [7]:
# 함수 추가 만드는 중

In [9]:
class SGD:
    def __init__(self,lr=1e-3):
        self.lr = lr
    
    def minimize(self, w, grads):
        for key in w.keys():
            w[key] -=  self.lr*grads[key]  

In [10]:
class Momentum:
    def __init__(self,lr=1e-3,m=0.9):
        self.lr = lr
        self.momentum = m
        self.v = None
    
    def minimize(self,w,grads):
        if self.v is None:
            self.v = {}
            for k,v in w.items():
                self.v[k] = np.zeros_like(v)
        for key in w.keys():
            self.v[key] = self.momentum + self.v[key] - self.lr*grads[key]
            w[key] += self.v[key]